# Задача
Дан ряд Тейлора для синусной интегральной функции:
$$
\text{Si}(x) = \sum_{n=0}^{\infty} \frac{(-1)^n x^{2n+1}}{(2n+1)(2n+1)!}
$$

### Нахождение отношения последующих членов ряда

Общий член ряда для $\text{Si}(x)$ имеет вид:

$$
a_n = \frac{(-1)^n x^{2n+1}}{(2n+1) \cdot (2n+1)!}
$$

Следующий член:

$$
a_{n+1} = \frac{(-1)^{n+1} x^{2n+3}}{(2n+3) \cdot (2n+3)!}
$$

Теперь вычислим отношение $q_n(x) = \frac{a_{n+1}}{a_n}$, показывая все преобразования шаг за шагом.

$$
q_n(x) = \frac{a_{n+1}}{a_n} = \frac{ \frac{(-1)^{n+1} x^{2n+3}}{(2n+3) \cdot (2n+3)!} }{ \frac{(-1)^n x^{2n+1}}{(2n+1) \cdot (2n+1)!} } = \frac{(-1)^{n+1} x^{2n+3}}{(2n+3) \cdot (2n+3)!} \cdot \frac{(2n+1) \cdot (2n+1)!}{(-1)^n x^{2n+1}}
$$

Сначала упростим знаки и степени $x$:

$$
= (-1)^{n+1} \cdot \frac{1}{(-1)^n} \cdot \frac{x^{2n+3}}{x^{2n+1}} \cdot \frac{(2n+1)}{(2n+3)} \cdot \frac{(2n+1)!}{(2n+3)!}
$$

$$
= (-1)^{(n+1) - n} \cdot x^{2} \cdot \frac{(2n+1)}{(2n+3)} \cdot \frac{(2n+1)!}{(2n+3)!} = -x^2 \cdot \frac{(2n+1)}{(2n+3)} \cdot \frac{(2n+1)!}{(2n+3)!}
$$

Теперь разберёмся с факториалами. Мы знаем, что:

$$
(2n+3)! = (2n+3) \cdot (2n+2) \cdot (2n+1)!
$$

Поэтому:

$$
\frac{(2n+1)!}{(2n+3)!} = \frac{(2n+1)!}{(2n+3) \cdot (2n+2) \cdot (2n+1)!} = \frac{1}{(2n+3) \cdot (2n+2)}
$$

(Здесь $(2n+1)!$ в числителе и знаменателе сокращается полностью, оставляя только обратные множители от $(2n+3)$ и $(2n+2)$.)

Подставляем это обратно:

$$
q_n(x) = -x^2 \cdot \frac{(2n+1)}{(2n+3)} \cdot \frac{1}{(2n+3) \cdot (2n+2)} = -x^2 \cdot \frac{(2n+1)}{(2n+3) \cdot (2n+3) \cdot (2n+2)}
$$

$$
= -x^2 \cdot \frac{(2n+1)}{(2n+2) \cdot (2n+3)^2}
$$

$$
= -\frac{x^2 (2n+1)}{(2n+2) (2n+3)^2}
$$

Таким образом, добавочный член $a_{n+1}$ можно получить как $a_{n+1} = q_n(x) \cdot a_n$, где $q_n(x)$ выражено через предыдущий член без полного перерасчёта факториалов. Это удобно для итеративных вычислений ряда, чтобы избежать больших чисел.

In [21]:
def ratio_series(x: float, n: int) -> float:
    """
    q_n(x) - отношения последующего члена к предыдущему
    :param x: аргумент
    :param n: индекс члена ряда
    """
    return -(
            x**2 * (2*n + 1)
    ) / (
            (2*n + 2) * (2*n + 3)**2
    )

In [22]:
def series(x: float, eps: float = 10 ** -6) -> float:
    """
    S_n(x) - реализация с использованием заранее вычисленного отношения последующего члена
    :param x: аргумент
    :param eps: точность
    """
    n = 0
    series_member_old = x # Рассчитанное значение для n = 0
    sum_series = x
    while True:
        n += 1
        series_member_new = series_member_old * ratio_series(x, n - 1)
        sum_series += series_member_new
        if abs(series_member_new - series_member_old) < eps: break
        series_member_old = series_member_new
    return sum_series

Вычисление суммы в предложенных преподавателем точках x = 0.8; x = 2; x = 3.2

In [23]:
for x in [0.8, 2, 3.2]: print(f'{series(x):6f}')

0.772096
1.605413
1.851401


Нативная реализация подсчета суммы (не оптимизированная)

In [24]:
import math

def native_series(x: float, eps: float = 10 ** -6) -> float:
    """
    S_n(x) - нативная реализация суммы
    :param x: аргумент
    :param eps: точность
    """
    n = 0
    series_member_old = x
    sum_series = x
    while True:
        n += 1
        series_member_new = ((-1)**n * x**(2*n + 1)) / ((2*n + 1) * math.factorial(2*n + 1))
        sum_series += series_member_new
        if abs(series_member_new - series_member_old) < eps: break
        series_member_old = series_member_new
    return sum_series

Сравнение моей реализации с нативной в предложенных точках

In [25]:
print('Моя\t\t\tНативная')
for x in [0.8, 2, 3.2]: print(f'{series(x):6f}\t{native_series(x):6f}')

Моя			Нативная
0.772096	0.772096
1.605413	1.605413
1.851401	1.851401


### Построение таблицы значений

In [32]:
import pandas

def tabulate(func, a: float = 0, b: float = 4, n: int = 5) -> list[tuple[float, float]]:
    """
    :param func: табулируемая функция
    :param a: начальная точка отрезка a < b
    :param b: конечная точка отрезка b > a
    :param n: количество интервалов
    :return: список пар (x, f(x))
    """
    h = (b - a) / n
    tab_x = [a]
    while len(tab_x) < n: tab_x.append(tab_x[-1] + h)
    tab_x.append(b)
    return [(x, func(x)) for x in tab_x]

data = tabulate(series)
x_values = [x for x, _ in data]
y_values = [f_x for _, f_x in data]

df = pandas.DataFrame([x_values, y_values], index=['x', 'f(x)'])
styled_df = df.style.format('{:.2f}')
styled_df

,0,1,2,3,4,5
x,0.00,0.80,1.60,2.40,3.20,4.00
f(x),0.00,0.77,1.39,1.75,1.85,1.76
